In [91]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
import jieba as jb
import pickle
from gensim.models import word2vec
from scipy import spatial

In [2]:
chats = pickle.load(open("chat.pkl", "rb"))

In [3]:
flat_list = [jb.lcut(item) for sublist in chats for item in sublist]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/9t/syxn77t51vnbj9ntx09wpzlr0000gn/T/jieba.cache
Loading model cost 1.224 seconds.
Prefix dict has been built succesfully.


In [4]:
wmodel = word2vec.Word2Vec(flat_list, size=250, sg=1)
wmodel.save("wmodel.w2v")

In [39]:
corpus = [" ".join(item) for item in flat_list]
vectorizer = CountVectorizer(max_features=7000,token_pattern='(?u)\\b\\w+\\b')
X = vectorizer.fit_transform(corpus)
word = vectorizer.get_feature_names()
counts = X.toarray()
# word = vectorizer.get_feature_names()  
# 获取每个词在该行（文档）中出现的次数
counts =  X.toarray()
transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf = transformer.fit_transform(X)
# tfidf.toarray()

In [101]:
pickle.dump(vectorizer, open("vectorizer", "wb"))
pickle.dump(transformer, open("tfidf", "wb"))

In [109]:
vectorizer = pickle.load(open("vectorizer", "rb"))
transformer = pickle.load(open("tfidf", "rb"))
wmodel = word2vec.Word2Vec.load("wmodel.w2v")
query = "要不要跟我約會？"
candidate_replies = {
    "興趣，喜好": "我的興趣是打羽毛球與打爵士鼓",
    "專長，擅長": "我的專長是打羽毛球，從高中開始打了9年",
    "程式語言，寫code": "我比較擅長Python與Ruby, 擅長的深度學習工具是PyTorch",
    "學歷，大學，研究所": "我大學念台大資管，目前繼續就讀台大資管所碩二",
    "論文，研究": "我的碩士論文主要研究Machine Reading Comprehension的技術",
    "工作經驗，實習": "大學時曾經在iCook實習一年擔任後端工程師, 碩一暑假在台達電擔任NLP工程師",
    "工作內容": "iCook時設計與建置API, 台達電時研究前沿的自動摘要技術",
    "自我介紹": "目前就讀台灣大學資訊管理學系碩二，研究所期間主要在研究Deep Learning NLP領域中的閱讀理解問題，實際實作過許多論文中的模型，並正在撰寫碩士論文目標投稿ACL 2019。工作經驗有在台達電子做過自動摘要技術的研究，以及大學部期間擁有一年的後端開發實習經驗，熟悉設計與實作基礎的網頁系統與API。"
}

In [110]:
def sent2vec(sent):
    sent_split = jb.lcut(sent)
    sent_blank = " ".join(sent_split)
    sent_tfidf = transformer.transform(vectorizer.transform([sent_blank])).toarray()[0]
    sent_vec = []
    for word in sent_split:
        if word in wmodel.wv.vocab:
            if word in vectorizer.vocabulary_:
                weight = sent_tfidf[vectorizer.vocabulary_[word]]
            else:
                weight = 0.05
            if len(sent_vec) == 0:
                sent_vec = weight * wmodel.wv[word]
            else:
                sent_vec += weight * wmodel.wv[word]
    return sent_vec / len(sent_split)

In [111]:
query_vec = sent2vec(query)
max_similarity = -1
thres = 0.0
cur_cand = "抱歉我不懂您的意思"
for cand in candidate_replies.keys():
    print(cand, ": ", end="")
    cand_vec = sent2vec(cand)
#     simi = np.dot(query_vec, cand_vec)
    simi = 1 - spatial.distance.cosine(query_vec, cand_vec)
    print(simi)
    if simi > max_similarity and simi > thres:
        max_similarity = simi
        cur_cand = cand
candidate_replies[cur_cand]

興趣，喜好 : 0.34394213557243347
專長，擅長 : 0.39824777841567993
程式語言，寫code : 0.36028310656547546
學歷，大學，研究所 : 0.3890741169452667
論文，研究 : 0.3212156295776367
工作經驗，實習 : 0.4011821150779724
工作內容 : 0.3849673271179199
自我介紹 : 0.3878204822540283


'大學時曾經在iCook實習一年擔任後端工程師, 碩一暑假在台達電擔任NLP工程師'